# Building A Chatbot

in this video will go over an example of how to design and impliment an simple LLM based chatbot this chat bot will able to have a conversation and remember previous intraction.

Note that this chatbot that will build will only use  of the language model to have a conversation .There are saveral other realated concept that you may be looking  fro:


1.Conversational Rag: Enable a chatbot experience over an external source of data.
2. Agent: Build a chatbot that can take action.

This video tutorial will cover the basics which will be helpful for those two or more advance topics.

In [13]:
! pip install -r requrements.txt

  Using cached langchain_chroma-0.2.5-py3-none-any.whl.metadata (1.1 kB)
Using cached langchain_chroma-0.2.5-py3-none-any.whl (12 kB)


In [ ]:
import os 
# from dotenv import load_dotenv
# load_dotenv()

# groq_api_key=os.getenv("Groq_key")
# groq_api_key

In [ ]:

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
# model= ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B42661BE10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B4267631D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage


model.invoke([HumanMessage(content="Hi,my name is krishna and ia am chief AI Engineer")])

AIMessage(content="Hello Krishna! It's a pleasure to meet you. That's a fantastic title!\n\nAs a chief AI engineer, what kind of projects are you working on these days? I'm always interested in learning more about the innovative things people are doing with AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 58, 'prompt_tokens': 21, 'total_tokens': 79, 'completion_time': 0.105454545, 'prompt_time': 0.00132173, 'queue_time': 0.249424739, 'total_time': 0.106776275}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d7db6aac-d49c-481a-b8e2-b4e4a9bd3099-0', usage_metadata={'input_tokens': 21, 'output_tokens': 58, 'total_tokens': 79})

In [5]:
from langchain_core.messages import AIMessage
model.invoke([HumanMessage(content="Hi,my name is krishna and ia am chief AI Engineer"),
              AIMessage(content="Hi Krishna, it's nice to meet you! That's a very impressive title. \n\nWhat kind of AI projects are you working on these days?  I'm always interested to hear about what people are doing in the field."),
              HumanMessage(content="hey what is my name and what is my designation")
              ])

AIMessage(content='You told me your name is Krishna and that you are a Chief AI Engineer!  \n\nIs there anything else I can help you with? 😊  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 91, 'total_tokens': 124, 'completion_time': 0.06, 'prompt_time': 0.00263538, 'queue_time': 0.247336059, 'total_time': 0.06263538}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--91ca4a60-25a3-4aa8-ae7e-327b92fd3075-0', usage_metadata={'input_tokens': 91, 'output_tokens': 33, 'total_tokens': 124})

###  Message History 

We can use a message History class to wrap up our model and make it statefull .This will keep track of input and output of the model ,and store them in some datastore.future intraction will then load those messages and pass them into the chain as part of the input .

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}


In [8]:
Response=with_message_history.invoke(
    [
        HumanMessage(content="Hi ,My name is Krishna and i am a python devloper")
    ],config=config
)

In [9]:
Response.content

"Hi Krishna, it's nice to meet you!\n\nThat's awesome that you're a Python developer.  \n\nWhat kind of projects are you working on these days? Do you have any particular areas of Python development that you're passionate about?\n"

In [11]:
Response=with_message_history.invoke(
    [
        HumanMessage(content="what is my name")
    ],config=config
)
Response.content

'You told me your name is Krishna earlier!  \n\nIs there something specific you want to know about your name, or are you just checking?  😄\n'

In [12]:
## Change the config---->Session id

config1={"configurable":{"session_id":"chat2"}}
Response=with_message_history.invoke(
    [
        HumanMessage(content="what is my name")
    ],config=config1
)
Response.content

"As an AI, I have no access to your personal information, including your name. If you'd like to tell me your name, I'd be happy to know!\n"

## Promp Templates

Prompt Templates Help to turn user information into a format that can work with . in this case the raw user input is a just message,which we are passing to the LLM.Lets now make that a bit more complicated.firstlets add in a system message with some custom instruction (But still taking message as input ) .Next we'll add in more input besides just the message.



In [13]:
from langchain_core.prompts import ChatPromptTemplate ,MessagesPlaceholder

prompt=ChatPromptTemplate.from_messages([
    ("system","you are helpfull assistent .Answer allthe question to the best of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is krishna")]})

AIMessage(content="Hello Krishna, it's nice to meet you! \n\nI'm ready to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 34, 'prompt_tokens': 31, 'total_tokens': 65, 'completion_time': 0.061818182, 'prompt_time': 0.00147471, 'queue_time': 0.250532459, 'total_time': 0.063292892}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--445e53fe-a29e-45b3-b655-93d35139349e-0', usage_metadata={'input_tokens': 31, 'output_tokens': 34, 'total_tokens': 65})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [20]:
config={"configurable":{"session_id":"chat3"}}
Response=with_message_history.invoke([
    HumanMessage(content="hi my name is krishna") 
],config=config)
Response.content

"Hello Krishna, it's nice to meet you! \n\nI'm happy to help with any questions you have. Just ask away! 😊 \n\nWhat can I do for you today?  \n\n"

In [ ]:
## Add more complexity
prompt=ChatPromptTemplate.from_messages([
    (
        "system",
        "you are a helpfull assistant . Answer all questions to the best of your ability in {language}"
    ),
    MessagesPlaceholder(variable_name="messages"),
])
chain=prompt|model

In [22]:
Response=chain.invoke({"messages":
                       [
                           HumanMessage(content="hi my name is krishna")
                       ],"language":"Hindi"})
Response.content

'नमस्ते कृष्ण! मैं आपकी मदद करने के लिए यहाँ हूँ। आपके कोई सवाल हैं? मैं अपनी पूरी कोशिश करूँगा कि जितनी अच्छी तरह से हो सके उत्तर दे सकूँ।  \n'

Lets now Wrap this more complicated chain in a message History class. This timebecouse there are multiple keys in the input ,we need to specify the correct key to use to save the chat History.

In [23]:
with_message_history=RunnableWithMessageHistory(
chain,
get_session_history,
input_messages_key="messages"
)

In [25]:
config={"configurable":{"session_id":"chat3"}}
Response=with_message_history.invoke(
    {
        'messages':[
            HumanMessage(content="Hi,I am Krishna")
        ],"language":"Hindi"
    },config=config
)
Response.content

'Namaste Krishna!  \n\nमुझे प्\u200dयारे कृष्\u200dण से मिलाना बहुत अच्छा लगा।  😊 \n\nआप कैसे हैं? \n'

In [27]:
Response=with_message_history.invoke(
    {
        'messages':[
            HumanMessage(content="what is my name")
        ],"language":"Hindi"
    },config=config
)
Response.content

'आपका नाम कृष्ण है। 😊  \n'

## Managing The Conversation History

One importent concepts tounderstand when building chatboats is how to manage conversation history .if left unmanaged the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therfore it is importent to add a step that limits the size of the messages you are passing in.

'trim_messages' helper to reduce how many messages we're sending to the model .The trimmer allow us to specify how may tokens we want ti keep .along with other parameter like if we want to always keep the system message and whether to allow partial messages. 

In [39]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
max_tokens=45,
strategy="last",
token_counter=model,
include_system=True,
allow_partial=False,
start_on="human"
)

messages=[

    SystemMessage(content="you are good assistent"),
    HumanMessage(content="hi i am a krishna"),
    AIMessage(content="hi!"),
    HumanMessage(content="i like vanila ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="what is 2+2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having Fun!"),
    AIMessage(content="yes!"),
    
]
trimmer.invoke(messages)

[SystemMessage(content='you are good assistent', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like vanila ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='what is 2+2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having Fun!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [40]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model

)
Response=chain.invoke(
    {

    "messages": messages + [ HumanMessage(content="what ice cream do i like")],
"language":"English"
}
)

Response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences! \n\nWhat's your favorite flavor? 😊🍦  \n\n\n\n\n"

In [41]:
Response=chain.invoke(
    {

    "messages": messages + [ HumanMessage(content="what math problem i asked")],
"language":"English"
}
)

Response.content

"You asked: 2 + 2 \n\n\n\nLet me know if you'd like to try another one! 😊  \n"

In [42]:
## Wrap this in the message History

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config={"configurable":{"session_id":"chat4"}}


Response=with_message_history.invoke(
    {

    "messages": messages + [ HumanMessage(content="what math problem i asked")],
"language":"English"
},
config=config
)

Response.content


'You asked "What is 2+2?"  😊 \n\n\n'